In [5]:
pip install sagemaker


Defaulting to user installation because normal site-packages is not writeable
  Using cached sagemaker-2.230.0-py3-none-any.whl (1.6 MB)
  Using cached schema-0.7.7-py2.py3-none-any.whl (18 kB)
     |████████████████████████████████| 11.3 MB 3.9 MB/s eta 0:00:01
  Using cached importlib_metadata-6.11.0-py3-none-any.whl (23 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached boto3-1.35.8-py3-none-any.whl (139 kB)
     |████████████████████████████████| 172 kB 21.7 MB/s eta 0:00:01
  Using cached jsonschema-4.23.0-py3-none-any.whl (88 kB)
  Using cached docker-7.1.0-py3-none-any.whl (147 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
     |████████████████████████████████| 14.0 MB 17.8 MB/s eta 0:00:01
  Using cached cloudpickle-2.2.1-py3-none-any.whl (25 kB)
  Using cached protobuf-4.25.4-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
  Using cached smdebug_rulesconfig-1.0.1-py2.py3-none-any.whl (2

In [13]:
import sagemaker
import boto3

#Connecting with AWS
#s3_client = boto3.client('s3', region_name='us-east-1')
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = "zamhouseprice"
print("Using the bucket", bucket)
region


Using the bucket zamhouseprice


'us-east-2'

In [2]:
#Import libraries
import pandas as pd
import numpy as np


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

### Get the data from the Drive

In [3]:
!gdown --id '1dHJT1AkJ-roBpfar-6trD6yfitNPhGNe'

zsh:1: command not found: gdown


In [4]:
df=pd.read_csv('/Users/usmanali/Uwindsor/AI/House_Price_Predict_by_Zameen.com/Flask_Server/Datasets/cleaned_data_for_model.csv')
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.head()

,property_type,price,location,city,baths,purpose,bedrooms,Area_in_Marla
0,Flat,10000000,G-10,Islamabad,2,For Sale,2,4.0
1,Flat,6900000,E-11,Islamabad,3,For Sale,3,5.6
2,House,16500000,G-15,Islamabad,6,For Sale,5,8.0
3,House,43500000,Bani Gala,Islamabad,4,For Sale,4,40.0
4,House,7000000,DHA Defence,Islamabad,3,For Sale,3,8.0


#### Defining the dependent and independent variables

In [5]:
x=df.drop(['price','location'],axis=1)
y=df.iloc[:,1:2]

In [6]:
x

,property_type,city,baths,purpose,bedrooms,Area_in_Marla
0,Flat,Islamabad,2,For Sale,2,4.0
1,Flat,Islamabad,3,For Sale,3,5.6
2,House,Islamabad,6,For Sale,5,8.0
3,House,Islamabad,4,For Sale,4,40.0
4,House,Islamabad,3,For Sale,3,8.0
...,...,...,...,...,...,...
99494,Flat,Karachi,3,For Sale,3,8.0
99495,House,Karachi,4,For Sale,3,8.0
99496,House,Karachi,3,For Sale,3,8.0
99497,House,Karachi,4,For Sale,4,14.0


In [7]:
y

,price
0,10000000
1,6900000
2,16500000
3,43500000
4,7000000
...,...
99494,7500000
99495,8800000
99496,14000000
99497,14000000


### Firstly we have to do the one-hot encoding to covert the categorical features into numerical

In [8]:
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0,1,3])],remainder='passthrough')
x=ct.fit_transform(x)

Split the data into train and test

In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

## Saving the data into S3 Bucket


In [10]:
trainX = pd.DataFrame(x_train)
trainX['label'] = y_train

testX = pd.DataFrame(x_test)
testX['label'] = y_test

In [11]:
# Creating the files
trainX.to_csv("x_trainV1.csv", index= False)
testX.to_csv("x_testV1.csv", index = False)

In [14]:
#Uploading the files into s3 bucket
sk_prefix = "sagemaker/house_price_prediction/sklearncontainer"
train_path = sess.upload_data(
    path = "x_testV1.csv", bucket = bucket, key_prefix= sk_prefix
)

test_path = sess.upload_data(
    path = "x_trainV1.csv", bucket= bucket, key_prefix= sk_prefix
)

print(train_path)
print(test_path)

s3://zamhouseprice/sagemaker/house_price_prediction/sklearncontainer/x_testV1.csv
s3://zamhouseprice/sagemaker/house_price_prediction/sklearncontainer/x_trainV1.csv


### Model Building
In which try all the Regression model and will see the results.

### Linear Regression

In [13]:
#Starting with linear regression
from sklearn.linear_model import LinearRegression
model=LinearRegression()

model.fit(x_train,y_train)
prediction=model.predict(x_test)

In [14]:
linear_model=model.score(x_test,y_test)
print("Linear Model Score:",linear_model)

Linear Model Score: 0.597120792107532


### Decision Tree

In [15]:
from sklearn import tree
dt_model=tree.DecisionTreeRegressor(criterion='squared_error',max_depth=3,max_leaf_nodes=5)
dt_model.fit(x_train,y_train)
dt_score=dt_model.score(x_test,y_test)
print("Decision Tree model score:",dt_score)


Decision Tree model score: 0.7358533641982425


### Random Forest

In [16]:
from sklearn.ensemble import RandomForestRegressor
random_forest_model=RandomForestRegressor(n_estimators=10,
                                          criterion='squared_error',
                                          max_depth=5,
                                          min_samples_leaf=5,)
random_forest_model.fit(x_train,y_train)
random_forest_score=random_forest_model.score(x_test,y_test)
print("Random Forest Score is:",random_forest_score)

/Users/usmanali/anaconda3/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Score is: 0.7991899152201849


KNN

In [17]:
from sklearn.neighbors import KNeighborsRegressor
knn_model=KNeighborsRegressor(n_neighbors=5,
                              weights='distance')
knn_model.fit(x_train,y_train)
knn_score=knn_model.score(x_test,y_test)
print("KNN score:",knn_score)

KNN score: 0.8310570639797328


In [18]:
from sklearn.ensemble import GradientBoostingRegressor
gradient_boost_model=GradientBoostingRegressor(learning_rate=1.0,
                                               n_estimators=20,
                                               max_depth=5,
                                               min_samples_leaf=5
                                               )
gradient_boost_model.fit(x_train,y_train)
gradient_boost_score=gradient_boost_model.score(x_test,y_test)
print("Gradient Boost Model Score:",gradient_boost_score)

/Users/usmanali/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Gradient Boost Model Score: 0.839670971786119


### XGBoost

In [19]:
import xgboost as xgb
xgb_model = xgb.XGBRegressor(objective="reg:linear", random_state=42)
xgb_model.fit(x_train,y_train)
xgb_score=xgb_model.score(x_test,y_test)
print("XG Boost score:",xgb_score)

XG Boost score: 0.8512254299744458


/Users/usmanali/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [00:34:57] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


### ADA Boost

In [20]:
from sklearn.ensemble import AdaBoostRegressor
adaboost_model=AdaBoostRegressor(n_estimators=20,
                                 learning_rate=0.1,
                                 loss='square')
adaboost_model.fit(x_train,y_train)
adaboost_score=adaboost_model.score(x_test,y_test)
print('ADA boost model score:',adaboost_score)

/Users/usmanali/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


ADA boost model score: 0.76313864456297


## So from here the best Accuracy Score that we got is from KNN and XG boost so we can any of them to deploy on any machine, so now I will try RandomSearch CV to get the best results from KNN only.

In [21]:
#Prediction
input_data=[['House','Wazir Town','Rawalpindi',4,'For Sale',3,165.9]]
input_data1=pd.DataFrame(input_data,columns=['property_type'	,'location',	'city'	,'baths'	,'purpose'	,'bedrooms',	'Area_in_Marla'])
ct=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0,1,2,4])],remainder='passthrough')
input_data2=ct.fit_transform(input_data1)

input_data2


array([[  1. ,   1. ,   1. ,   1. ,   4. ,   3. , 165.9]])

In [25]:
#Save the model
import pickle
pickle.dump(knn_model, open('House_Price_Prediction_model.pkl', 'wb'))

In [24]:
pickled_model = pickle.load(open('House_Price_Prediction_with_KNN.pkl', 'rb'))
pickled_model.predict(x_test)

array([[11260000.],
       [ 5500000.],
       [   23800.],
       ...,
       [ 7700000.],
       [ 4160000.],
       [ 6340000.]])

In [ ]:
x_test